# OpenCV example

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import subprocess

In [ ]:
def display_img(img):
    if len(img.shape) == 3:
        img2 = img[:,:,::-1]
        plt.imshow(img2)
    elif len(img.shape) == 2:
        plt.imshow(img, cmap='gray', vmin=0, vmax=255)  

def sub_image(img, point1, point2):
    img_height = img.shape[0]
    img_width = img.shape[1]
    print(img_height, img_width, point1, point2)
    if point1[1] > img_height:
        raise Exception("Sub image outside of bounds")
    if point2[1] > img_height:
        raise Exception("Sub image outside of bounds")
    if point1[0] > img_width:
        raise Exception("Sub image outside of bounds")
    if point2[0] > img_width:
        raise Exception("Sub image outside of bounds")
    if len(img.shape) == 2:
        return img[point1[1]:point2[1], point1[0]:point2[0]]
    elif len(img.shape) == 3:
        return img[point1[1]:point2[1], point1[0]:point2[0],:]

def tl_point(point, width, height):
    return (point[0] - round(width/2.), point[1] - round(height/2.))

def br_point(point, width, height):
    return (point[0] - round(width/2.) + width, point[1] - round(height/2.) + height)

def draw_rectangle(img, point1, point2, color, thickness):
    tl = point1
    tr = (point2[0], point1[1])
    br = point2
    bl = (point1[0], point2[1])
    cv.line(img, tl, tr, color, thickness)
    cv.line(img, tr, br, color, thickness)
    cv.line(img, br, bl, color, thickness)
    cv.line(img, bl, tl, color, thickness)

# Inputs

In [ ]:
subprocess.run(["cp", "../img/image01.jpg", "."])
subprocess.run(["cp", "../img/image02.jpg", "."])
subprocess.run(["cp", "../img/image03.jpg", "."])

In [ ]:
img = cv.imread('image01.jpg')

In [ ]:
display_img(img)

In [ ]:
WIDTH = 500
HEIGHT = 500

X_OFFSET = 50
Y_OFFSET = 70

In [ ]:
center1 = (round(img.shape[1]/2), round(img.shape[0]/2))
tl1 = tl_point(center1, WIDTH, HEIGHT)
br1 = br_point(center1, WIDTH, HEIGHT)
print('center 1', center1)
print('TL 1', tl1)
print('BR 1', br1)

center2 = (center1[0] + X_OFFSET, center1[1] + Y_OFFSET)
tl2 = tl_point(center2, WIDTH, HEIGHT)
br2 = br_point(center2, WIDTH, HEIGHT)
print('center 2', center2)
print('TL 2', tl2)
print('BR 2', br2)

In [ ]:
d_img = img.copy()

In [ ]:
draw_rectangle(d_img, tl1, br1, (255, 0, 0), 10)
draw_rectangle(d_img, tl2, br2, (0, 0, 255), 10)

In [ ]:
display_img(d_img)

In [ ]:
img1 = sub_image(img, tl1, br1)
display_img(img1)

In [ ]:
img2 = sub_image(img, tl2, br2)
display_img(img2)

# Edge Detection

In [ ]:
gray = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
edges = cv.Canny(img1, 100, 200)

plt.subplot(121),display_img(img1)
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),display_img(edges)
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

# Template Matching (cross correlation)

In [ ]:
def edges(img, n= 6):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    kernel = np.ones((n, n),np.float32)/(n*n)
    gray = cv.filter2D(gray,-1,kernel)
    return cv.Canny(gray, 100, 200)

In [ ]:
# img2

In [ ]:
# All the 6 methods for comparison in a list
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
 'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
 
t_img = img1.copy()
# img_a = edges(img1, n=6)
img_a = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
# img_b = edges(img2, n=6)
img_b = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
draw_rectangle(t_img, tl1, br1, (255, 0, 0), 10)

print('TARGET:', center1)
for meth in methods:
    print("Method:", meth)
    method = eval(meth)
 
    # Apply template Matching
    res = cv.matchTemplate(img_a, img_b, method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    print("MAX", max_val, max_loc)
    draw_rectangle(
        t_img,
        (round(max_loc[0]-WIDTH/2), round(max_loc[1]-HEIGHT/2)),
        (round(max_loc[0]+WIDTH/2), round(max_loc[1]+HEIGHT/2)),
        (0, 255, 0), 10)
    print("MIN", min_val, min_loc)
    draw_rectangle(
        t_img,
        (round(min_loc[0]-WIDTH/2), round(min_loc[1]-HEIGHT/2)),
        (round(min_loc[0]+WIDTH/2), round(min_loc[1]+HEIGHT/2)),
        (0, 0, 255), 10)
#     display_img(res)
#     print("MIN")
#     print(min_val)
#     print(min_loc)
 
#  # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
#  if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
#  top_left = min_loc
#  else:
#  top_left = max_loc
#  bottom_right = (top_left[0] + w, top_left[1] + h)
 
#  cv.rectangle(img,top_left, bottom_right, 255, 2)
 
#  plt.subplot(121),plt.imshow(res,cmap = 'gray')
#  plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
#  plt.subplot(122),plt.imshow(img,cmap = 'gray')
#  plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
#  plt.suptitle(meth)
 
#  plt.show()

display_img(t_img)

In [ ]:
# display_img(img_b)

# SIFT

In [ ]:
img_in = sub_image(cv.imread('image03.jpg'), (100, 100), (1000, 1000))
# img_in = img2.copy()

In [ ]:
display_img(img_in)

In [ ]:
sift = cv.SIFT_create()
gray = cv.cvtColor(img_in, cv.COLOR_BGR2GRAY)
# gray = edges(img3, n=3)
kp = sift.detect(gray, None)
 
t_img = img_in.copy()
cv.drawKeypoints(img_in, kp, t_img)
 
display_img(t_img)

In [ ]:
print(len(kp))

In [ ]:
idx = 0 
for p in kp:
    idx += 1
    if idx > 10:
        break
    print("===================================================")
    print("point", p.pt, "angle", p.angle)
    print("octave", p.octave, "class", p.class_id, "response", p.response, "size", p.size)

# ORB

In [ ]:
orb = cv.ORB_create()

gray = cv.cvtColor(img3, cv.COLOR_BGR2GRAY)
# gray = edges(img3, n=4)

kp, des = orb.compute(img3, None)
 
t_img = img3.copy()
cv.drawKeypoints(t_img, kp, t_img)
 
display_img(t_img)

In [ ]:
print(len(kp))

In [ ]:

# # Initiate ORB detector
# orb = cv.ORB_create()
 
# # find the keypoints with ORB
# kp = orb.detect(img,None)
 
# # compute the descriptors with ORB
# kp, des = orb.compute(img, kp)
 
# # draw only keypoints location,not size and orientation
# img2 = cv.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
# plt.imshow(img2), plt.show()

In [ ]:
# display_img(img)

In [ ]:
# display_img(img2)